In [91]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

%matplotlib inline

In [92]:
path_to_file = "shakespeare.txt"

In [93]:
text = open(path_to_file, 'r').read()

In [94]:
print(text[:500])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bu


In [95]:
vocab = sorted(set(text))

In [96]:
char_to_ind = {char: ind for ind, char in enumerate(vocab)}

In [97]:
char_to_ind['H']

33

In [98]:
ind_to_char = np.array(vocab)

In [99]:
encode_text = np.array([char_to_ind[c] for c in text])

In [100]:
encode_text

array([ 0,  1,  1, ..., 30, 39, 29])

In [101]:
seq_len = 120

In [102]:
total_num_seq = len(text) // (seq_len+1)

In [103]:
total_num_seq

45005

In [104]:
char_dataset = tf.data.Dataset.from_tensor_slices(encode_text)

In [105]:
sequence = char_dataset.batch(seq_len+1, drop_remainder=True)

In [106]:
sequence

<BatchDataset element_spec=TensorSpec(shape=(121,), dtype=tf.int32, name=None)>

In [107]:
def create_seq(seq):
    input_text = seq[:-1]
    target_text = seq[1:]
    return input_text, target_text

In [108]:
dataset = sequence.map(create_seq)

In [109]:
for input_txt, target_txt in dataset.take(1):
    print(input_txt.numpy())
    print("".join(ind_to_char[input_txt.numpy()]))
    print('\n')
    print(target_txt.numpy())
    print("".join(ind_to_char[target_txt.numpy()]))

[ 0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0
  1  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74
  1 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45
 63 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74
 60  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75]

                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But


[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0  1
  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74  1
 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45 63
 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74 60
  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75  1]
                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But 


In [110]:
batch_size = 128

In [111]:
buffer_size = 10000

dataset = dataset.shuffle(buffer_size=buffer_size).batch(batch_size, drop_remainder=True)

In [112]:
vocab_size = len(vocab)

In [113]:
embed_dim = 64

In [114]:
rnn_neurons = 1026

In [115]:
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [121]:
def sparse_cat (y_true, y_pred):
    return sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

In [117]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense

In [132]:
def create_model (vocab_size, embed_dim, rnn_neurons, batch_size):
    
    model = Sequential()
    
    model.add(Embedding(vocab_size, embed_dim, batch_input_shape=[batch_size, None]))
    model.add(GRU(rnn_neurons, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'))
    
    model.add(Dense(vocab_size))
    
    model.compile('adam', loss=sparse_cat)
    
    return model
    

In [133]:
model = create_model(vocab_size=vocab_size, embed_dim=embed_dim, rnn_neurons=rnn_neurons, batch_size=batch_size)

In [134]:
model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_14 (Embedding)    (128, None, 64)           5376      
                                                                 
 gru_10 (GRU)                (128, None, 1026)         3361176   
                                                                 
 dense_3 (Dense)             (128, None, 84)           86268     
                                                                 
Total params: 3,452,820
Trainable params: 3,452,820
Non-trainable params: 0
_________________________________________________________________


In [136]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)

In [137]:
sample_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)

In [138]:
sample_indices = tf.squeeze(sample_indices, axis=1).numpy()

In [139]:
epochs = 30

In [140]:
model.fit(dataset, epochs=epochs)

Epoch 1/30
 10/351 [..............................] - ETA: 9:54 - loss: 4.8357

KeyboardInterrupt: 

In [141]:
from tensorflow.keras.models import load_model

In [142]:
model = create_model(vocab_size, embed_dim, rnn_neurons, batch_size=1)

model.load_weights('shakespeare_gen.h5')

model.build(tf.TensorShape([1,None]))

In [143]:
def generate_text(model, start_seed, gen_size=500, temp=1.0):

  num_gen = gen_size

  input_eval = [char_to_ind[s] for s in start_seed] 

  input_eval = tf.expand_dims(input_eval, 0)

  text_generated = []

  temperature = temp

  model.reset_states()

  for i in range(num_gen):

    predictions = model(input_eval)

    predictions = tf.squeeze(predictions, 0)

    predictions = predictions / temperature

    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

    input_eval = tf.expand_dims([predicted_id],0)

    text_generated.append(ind_to_char[predicted_id])

  return (start_seed + "".join(text_generated))

In [144]:
print(generate_text(model, 'JULIET', gen_size=1000))

JULIETHXTERNINA. Andu-host.
  CASSIO. I did see her prize!
  PAULINA. Some meneather, headed tried his journaturners!             [ALL AENTON
  VALENTINE. Indeed, I am bound.
  ANTONIO. Yields was His sphere was so
    To signify to Angelo; whose honour here.
  PEDANT. Go, she, my noble lord,
    Hath came intoin'd to the recreahion.
  PETER. So; he does tend to Padua ake
    An archin charge, but not unto the worm;
    That is no stubborn shore in that same keen, provost, my brother's part.
  
             They knock att me in the wit. Down. come away,
    is that one, Iago.  
    Your back of his sounding hume, will you go about
    Anlust, and what you have done she    Had not the twenty or the apparition
    This call'st to theretor, and this shall express
    The anguish of contract be.              Exeunt

                  Enter ALIESS with three ot be.
    What libertite will, I have no form,
    And I will whip him dead.
  DEMETRIUS. And I the Emperor  Hot. Only agone, hath he